In [1]:
# Configure the necessary Spark environment
import os
import sys

spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "/python")

# Add the py4j to the path.
# You may need to change the version number to match your install
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.8.2.1-src.zip'))

# Initialize PySpark to predefine the SparkContext variable 'sc'
#execfile(os.path.join(spark_home, 'python/pyspark/shell.py'))

In [58]:
from pyspark.sql import SparkSession, SQLContext
import json


In [3]:
spark = SparkSession.builder.appName('Basic').getOrCreate()

In [8]:
dbconfig = {"url": "jdbc:postgresql://localhost/qxedb",
            "dbtable": "gateway_eventnotification",
            "user": "hermes",
            "password": "mysecret",
            "driver": "org.postgresql.Driver"}

In [9]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /work/ericr/spark/sparkdev/postgresql.jar pyspark-shell'


In [10]:
fdf = spark.read.jdbc(url="jdbc:postgresql://localhost/qxedb", 
                      table="gateway_eventnotification",
                      properties=dbconfig)

In [69]:
rdd = fdf.select("payload")

In [75]:
nrdd = rdd.rdd.map(lambda r: json.dumps(r[0]))

In [76]:
sqlcontext = SQLContext(spark.sparkContext)

In [84]:
sqlcontext.read.json(nrdd, multiLine=True).show()

+--------------------+
|     _corrupt_record|
+--------------------+
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"events\": [{\...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"events\": [{\...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
+--------------------+
only showing top 20 rows



In [83]:
js.show()

+--------------------+
|     _corrupt_record|
+--------------------+
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"events\": [{\...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"events\": [{\...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
|"{\"links\": [{\"...|
+--------------------+
only showing top 20 rows



In [56]:
for x in rdd.take(4):
    print(x)

Row(payload='{"links": [{"rel": "eventsByCategory", "href": "/cdm/telemetry/v1/eventing/events/supply"}], "events": [{"dateTime": "2001-02-19T19:01:42.0Z", "eventDetail": {"t4": 19780000, "brand": "HP", "isSetup": "false", "isTrial": "false", "version": "1.0.0", "midLabel": "00000000000000", "eventCode": "17.99.32", "isRefilled": "false", "supplyType": 1, "installDate": "2018-08-13T00:00:00.0Z", "eventTrigger": "usedConsumable", "manufacturer": "HP", "supplyRegion": 4, "inkLevelGauge": 10514988, "printerRegion": 4, "supplyFamilyId": 37, "inkFillCapacity": 11300000, "manufactureDate": "2018-06-04T00:00:00.0Z", "triggerCategory": "supplyChanged", "supplyMarkedEmpty": "false", "supplyModelNumber": 1, "consumableUniqueId": "000000000000000045f0ff19d100088c", "totalInkUsageGauge": 785011, "consumableLabelCode": "M", "inkLevelSensorValue": 54, "selectabilityNumber": "912XL", "measuredQuantityState": "ok", "consumableSerialNumber": "560256648", "postStartupSupplyVolume": 0, "isInstantInkSubsc

In [61]:
for x in nrdd.take(4):
    print(x)

["{\"links\": [{\"rel\": \"eventsByCategory\", \"href\": \"/cdm/telemetry/v1/eventing/events/supply\"}], \"events\": [{\"dateTime\": \"2001-02-19T19:01:42.0Z\", \"eventDetail\": {\"t4\": 19780000, \"brand\": \"HP\", \"isSetup\": \"false\", \"isTrial\": \"false\", \"version\": \"1.0.0\", \"midLabel\": \"00000000000000\", \"eventCode\": \"17.99.32\", \"isRefilled\": \"false\", \"supplyType\": 1, \"installDate\": \"2018-08-13T00:00:00.0Z\", \"eventTrigger\": \"usedConsumable\", \"manufacturer\": \"HP\", \"supplyRegion\": 4, \"inkLevelGauge\": 10514988, \"printerRegion\": 4, \"supplyFamilyId\": 37, \"inkFillCapacity\": 11300000, \"manufactureDate\": \"2018-06-04T00:00:00.0Z\", \"triggerCategory\": \"supplyChanged\", \"supplyMarkedEmpty\": \"false\", \"supplyModelNumber\": 1, \"consumableUniqueId\": \"000000000000000045f0ff19d100088c\", \"totalInkUsageGauge\": 785011, \"consumableLabelCode\": \"M\", \"inkLevelSensorValue\": 54, \"selectabilityNumber\": \"912XL\", \"measuredQuantityState\": 

In [77]:
t = nrdd.take(4)

In [79]:
type(t[0])

str

In [80]:
t[0]

'"{\\"links\\": [{\\"rel\\": \\"eventsByCategory\\", \\"href\\": \\"/cdm/telemetry/v1/eventing/events/supply\\"}], \\"events\\": [{\\"dateTime\\": \\"2001-02-19T19:01:42.0Z\\", \\"eventDetail\\": {\\"t4\\": 19780000, \\"brand\\": \\"HP\\", \\"isSetup\\": \\"false\\", \\"isTrial\\": \\"false\\", \\"version\\": \\"1.0.0\\", \\"midLabel\\": \\"00000000000000\\", \\"eventCode\\": \\"17.99.32\\", \\"isRefilled\\": \\"false\\", \\"supplyType\\": 1, \\"installDate\\": \\"2018-08-13T00:00:00.0Z\\", \\"eventTrigger\\": \\"usedConsumable\\", \\"manufacturer\\": \\"HP\\", \\"supplyRegion\\": 4, \\"inkLevelGauge\\": 10514988, \\"printerRegion\\": 4, \\"supplyFamilyId\\": 37, \\"inkFillCapacity\\": 11300000, \\"manufactureDate\\": \\"2018-06-04T00:00:00.0Z\\", \\"triggerCategory\\": \\"supplyChanged\\", \\"supplyMarkedEmpty\\": \\"false\\", \\"supplyModelNumber\\": 1, \\"consumableUniqueId\\": \\"000000000000000045f0ff19d100088c\\", \\"totalInkUsageGauge\\": 785011, \\"consumableLabelCode\\": \\"M\

In [70]:
rdd_t = rdd.take(4)

In [71]:
type(rdd_t[0])

pyspark.sql.types.Row

In [74]:
json.dumps(rdd_t[0][0])

'"{\\"links\\": [{\\"rel\\": \\"eventsByCategory\\", \\"href\\": \\"/cdm/telemetry/v1/eventing/events/supply\\"}], \\"events\\": [{\\"dateTime\\": \\"2001-02-19T19:01:42.0Z\\", \\"eventDetail\\": {\\"t4\\": 19780000, \\"brand\\": \\"HP\\", \\"isSetup\\": \\"false\\", \\"isTrial\\": \\"false\\", \\"version\\": \\"1.0.0\\", \\"midLabel\\": \\"00000000000000\\", \\"eventCode\\": \\"17.99.32\\", \\"isRefilled\\": \\"false\\", \\"supplyType\\": 1, \\"installDate\\": \\"2018-08-13T00:00:00.0Z\\", \\"eventTrigger\\": \\"usedConsumable\\", \\"manufacturer\\": \\"HP\\", \\"supplyRegion\\": 4, \\"inkLevelGauge\\": 10514988, \\"printerRegion\\": 4, \\"supplyFamilyId\\": 37, \\"inkFillCapacity\\": 11300000, \\"manufactureDate\\": \\"2018-06-04T00:00:00.0Z\\", \\"triggerCategory\\": \\"supplyChanged\\", \\"supplyMarkedEmpty\\": \\"false\\", \\"supplyModelNumber\\": 1, \\"consumableUniqueId\\": \\"000000000000000045f0ff19d100088c\\", \\"totalInkUsageGauge\\": 785011, \\"consumableLabelCode\\": \\"M\